# Helper

In [ ]:
import pickle
import spacy
import json
import numpy as np

from keras.models import load_model
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model, load_model

In [34]:
class Embedder(object):
    def __init__(self, max_length=100):
        self.max_length = max_length
        print("Loading spacy...")
        self.nlp = spacy.load('en')
        print("...done")

    def embed(self, text):
        return self._pad(self._vectors(text))

    def shape(self):
        return (self.max_length, 300)

    def _vectors(self, text):
        doc = self.nlp(text)
        vectors = []
        for token in doc:
            vectors.append(token.vector)
        return vectors

    def _pad(self, vectors):
        vector_dim = len(vectors[0])
        sequence = np.zeros((self.max_length, vector_dim))
        for i, vector in enumerate(vectors):
            if i == self.max_length:
                break
            sequence[i] = vector
        return sequence

class Predictor(object):
    def __init__(self, embedder, model_path, intent_names_path):
        self.model = load_model(model_path)
        self.intent_names = pickle.load(open(intent_names_path, "rb"))
        self.embedder = embedder

    def predict(self, text):
        proba = self._predict_proba(text)
        return self.intent_names[np.argmax(proba)]

    def _predict_proba(self, text):
        embedding = self.embedder.embed(text)
        return self.model.predict(np.array([embedding]))

def predict(model, text):
    embedding = embedder.embed(text)
    return model.predict(np.array([embedding]))


def translate(prediction, intent_names):
    return intent_names[np.argmax(prediction)]


def suggest(text):
    prediction = predict(loaded_model, text)
    return translate(prediction, loaded_intent_names)    

Loading spacy...
...done


# Prepare Data

In [41]:
def load_data(data_dir, limit=0):
    examples = []
    intent_names = {}
    with open(data_dir + '/01.json') as data_file:
        data = json.load(data_file)
    for i, intent in enumerate(data):
        intent_names[i] = intent["prompt"]
        for message in intent["response"]:
            examples.append((message, i))
    np.random.shuffle(examples)
    if limit >= 1:
        examples = examples[:limit]
    messages, intents = zip(*examples)
    return examples, intent_names

def dummy_encode(array, num_classes=None):
    array = np.array(array)
    if num_classes is None:
        num_classes = max(array) + 1
    result = np.zeros((len(array), num_classes))
    result[np.arange(len(array)), array] = 1
    return result

def create_dataset(type):
    if type == 'train':
        data_dir = 'data/train'
    elif type == 'dev':
        data_dir = 'data/dev'
    examples, intent_names = load_data(data_dir)
    X = []
    y = []
    for example in examples:
        message, intent = example[0], example[1]
        X.append(embedder.embed(message))
        y.append(intent)
    return np.array(X), dummy_encode(np.array(y)), intent_names

In [43]:
X_train, y_train, intent_names_train = create_dataset('train')
X_dev, y_dev, intent_names_dev = create_dataset('dev')
X, y, intent_names = X_train, y_train, intent_names_train

# Create CNN

In [58]:
def build_model(input_shape):
    sequences = Input(shape=input_shape)
    x = Conv1D(400, 5, activation='relu')(sequences)
    x = MaxPooling1D(2)(x)
    x = Conv1D(400, 3, activation='relu')(x)
    x = MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dense(400, activation='relu')(x)
    preds = Dense(len(intent_names), activation='softmax')(x)
    model = Model(input=sequences, output=preds)
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

def save_model(model, model_name):
    model.save('output/' + model_name)

# Intents

In [ ]:
pickle.dump(intent_names, open("output/intent_names.p", "wb"))
loaded_intent_names = pickle.load(open("output/intent_names.p", "rb"))

# Model 1

In [70]:
model = build_model((X.shape[1], X.shape[2]))
history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), nb_epoch=40, batch_size=256)
save_model(model, 'mymodel')
embedder = Embedder()

Train on 26408 samples, validate on 8794 samples
Epoch 1/40
26408/26408 [==============================] - 11s - loss: 4.5642 - acc: 0.0526 - val_loss: 3.8094 - val_acc: 0.1071
Epoch 2/40
26408/26408 [==============================] - 10s - loss: 3.0429 - acc: 0.1893 - val_loss: 2.9008 - val_acc: 0.2319
Epoch 3/40
26408/26408 [==============================] - 10s - loss: 2.1049 - acc: 0.3804 - val_loss: 1.7825 - val_acc: 0.4775
Epoch 4/40
26408/26408 [==============================] - 10s - loss: 1.4156 - acc: 0.5802 - val_loss: 1.2680 - val_acc: 0.6257
Epoch 5/40
26408/26408 [==============================] - 10s - loss: 1.0044 - acc: 0.7371 - val_loss: 0.7410 - val_acc: 0.8410
Epoch 6/40
26408/26408 [==============================] - 10s - loss: 0.7395 - acc: 0.8260 - val_loss: 0.5572 - val_acc: 0.8780
Epoch 7/40
26408/26408 [==============================] - 10s - loss: 0.6071 - acc: 0.8635 - val_loss: 0.4789 - val_acc: 0.8947
Epoch 8/40
26408/26408 [==============================]

In [72]:
loaded_model = load_model('output/mymodel')
predictor = Predictor(embedder, 'output/mymodel', 'output/intent_names.p')

In [77]:
print(predictor.predict('i want to pay with by credit card'),
predictor.predict('can i buy by post'),
predictor.predict('i would pay by dollars'),
predictor.predict('can i am a pay with credit card'))

Sag: Ich möchte mit Kreditkarte bezahlen Sag: Ich möchte mit Visa bezahlen Sag: Ich möchte mit Dollars bezahlen Sag: Ich möchte mit Kreditkarte bezahlen


In [27]:
model3 = build_model((X.shape[1], X.shape[2]))
history = model2.fit(X_train, y_train, nb_epoch=50, batch_size=512)

Epoch 1/50
43862/43862 [==============================] - 7s - loss: 0.5454 - acc: 0.9100     
Epoch 2/50
43862/43862 [==============================] - 6s - loss: 0.5310 - acc: 0.9124     
Epoch 3/50
43862/43862 [==============================] - 6s - loss: 0.5208 - acc: 0.9139     
Epoch 4/50
43862/43862 [==============================] - 6s - loss: 0.5124 - acc: 0.9173     
Epoch 5/50
43862/43862 [==============================] - 6s - loss: 0.5016 - acc: 0.9182     
Epoch 6/50
43862/43862 [==============================] - 6s - loss: 0.4867 - acc: 0.9234     
Epoch 7/50
43862/43862 [==============================] - 6s - loss: 0.4827 - acc: 0.9245     
Epoch 8/50
43862/43862 [==============================] - 7s - loss: 0.4700 - acc: 0.9284     
Epoch 9/50
43862/43862 [==============================] - 6s - loss: 0.4606 - acc: 0.9308     
Epoch 10/50
43862/43862 [==============================] - 6s - loss: 0.4558 - acc: 0.9310     
Epoch 11/50
43862/43862 [========================

In [29]:
save_model(model3)

In [30]:
predictor = Predictor(embedder, 'output/mymodel3', 'output/intent_names.p')
predictor.predict('can i buy by post')

'Sag: Ich warte auf das bestellte Essen'

In [33]:
predictor.predict('i want pay by euros')

'Sag: Ich möchte nächste Woche abreisen'

In [78]:
modelF = build_model((X.shape[1], X.shape[2]))
history = model.fit(X_train, y_train, validation_data=(X_dev, y_dev), nb_epoch=100, batch_size=256)
def save_model(model):
    model.save('output/mymodelF')

save_model(model)
pickle.dump(intent_names, open("output/intent_names.p", "wb"))
loaded_model = load_model('output/mymodelF')
loaded_intent_names = pickle.load(open("output/intent_names.p", "rb"))


def predict(model, text):
    embedding = embedder.embed(text)
    return model.predict(np.array([embedding]))


def translate(prediction, intent_names):
    return intent_names[np.argmax(prediction)]


def suggest(text):
    prediction = predict(loaded_model, text)
    return translate(prediction, loaded_intent_names)

embedder = Embedder()
predictor = Predictor(embedder, 'output/mymodelF', 'output/intent_names.p')
print(predictor.predict('i want to pay with by credit card'),
predictor.predict('can i buy by post'),
predictor.predict('i would pay by dollars'),
predictor.predict('can i am a pay with credit card'))

Train on 26408 samples, validate on 8794 samples
Epoch 1/100
26408/26408 [==============================] - 10s - loss: 0.0693 - acc: 0.9817 - val_loss: 0.1000 - val_acc: 0.9777
Epoch 2/100
26408/26408 [==============================] - 10s - loss: 0.0721 - acc: 0.9810 - val_loss: 0.0965 - val_acc: 0.9765
Epoch 3/100
26408/26408 [==============================] - 10s - loss: 0.0762 - acc: 0.9808 - val_loss: 0.1070 - val_acc: 0.9784
Epoch 4/100
26408/26408 [==============================] - 10s - loss: 0.0605 - acc: 0.9824 - val_loss: 0.1020 - val_acc: 0.9766
Epoch 5/100
26408/26408 [==============================] - 10s - loss: 0.0927 - acc: 0.9784 - val_loss: 0.1097 - val_acc: 0.9724
Epoch 6/100
26408/26408 [==============================] - 10s - loss: 0.0506 - acc: 0.9854 - val_loss: 0.1097 - val_acc: 0.9752
Epoch 7/100
26408/26408 [==============================] - 10s - loss: 0.0527 - acc: 0.9846 - val_loss: 0.4750 - val_acc: 0.9182
Epoch 8/100
26408/26408 [=======================

# Use Model on ST Data

In [84]:
kaldi = open('kaldi', 'r')
out = open('out', 'a')
for line in kaldi:
    out.write(predictor.predict(line) + '\n')
    
kaldi.close()
out.close()